In [1]:
import tensorflow as tf
import numpy as np

In [5]:
def get_text_data():
    with open("data/shakespeare.txt","r") as text_file:
        data=text_file.read()
        data=data.replace('\n','')
        data=data.replace('$','')
    lis=list(set(data))    
    total_char,unique_char=len(data),len(lis)
    print ('data has %d characters, %d unique.' %(total_char, unique_char))
    char_to_ix = { ch:i for i,ch in enumerate(lis) }
    ix_to_char = { i:ch for i,ch in enumerate(lis) }
    return total_char,unique_char,char_to_ix,ix_to_char,data

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


In [6]:
total_chars,unique_chars,char_id,id_char,data=get_text_data()

data has 1075393 characters, 63 unique.


In [7]:
print(char_id)

{'C': 0, ',': 1, 'Y': 2, "'": 3, 'm': 4, 'o': 5, 's': 6, 'D': 7, 'X': 8, 'y': 9, 'q': 10, 'i': 11, 'Z': 12, 'G': 13, 'J': 14, 'W': 15, '?': 16, 'N': 17, 'A': 18, ';': 19, 'T': 20, 'K': 21, 'h': 22, 'p': 23, 'u': 24, 'R': 25, 'O': 26, 'I': 27, 'c': 28, 'f': 29, 'U': 30, 'n': 31, 'b': 32, 'M': 33, '.': 34, 'Q': 35, ' ': 36, 'w': 37, ':': 38, 'F': 39, 'r': 40, 'P': 41, 'g': 42, 'B': 43, 'l': 44, 'E': 45, 'e': 46, '&': 47, 'H': 48, 'x': 49, 'v': 50, 'k': 51, 'z': 52, 'S': 53, 'V': 54, 't': 55, 'd': 56, 'a': 57, '!': 58, 'j': 59, '-': 60, 'L': 61, '3': 62}


In [8]:
# def generate_next_batch(seq_len,seed):
#     a=np.zeros((seq_len,1,unique_chars))
#     b=np.zeros((seq_len,unique_chars))
#     for t in range(seq_len):
#         a[t][0][char_id[data[t]]]=1
#         b[t][char_id[data[t+1]]]=1
#     return a,b,char_id[data[seed]]

In [9]:
def generate_next_batch(seq_length,p):
    inputs = [char_id[ch] for ch in data[p:p+seq_length]]
    targets = [char_id[ch] for ch in data[p+1:p+seq_length+1]]
    inputs=np.array(inputs).reshape(seq_length,1)
    targets=np.array(targets).reshape(seq_length,1)
    return inputs,targets

In [10]:
#test generator function here
def generator(seed_i,n):
    x=np.zeros((1,1))
    x[0][0]= seed_i
    ixes=[]
    for t in range(n):
        p=sess.run(prediction,{X:x})
        ix = np.random.choice(range(unique_chars), p=p.ravel())
        x[0][0]=ix
        ixes.append(ix)
    return ixes

In [23]:
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 0.1
n_neurons = 300
n_layers = 1 #3

In [24]:
def build_rnn(x):
        with tf.name_scope('rnn'):
            cell= tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
            outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
            return outputs,states

In [25]:
reset_graph()

X=tf.placeholder(tf.float32,[None,1])
Y=tf.placeholder(tf.float32,[None,1])
X=tf.cast(X,tf.int32)
X_onehot=tf.one_hot(X,unique_chars)
Y=tf.cast(Y,tf.int32)
Y_onehot=tf.one_hot(Y,unique_chars)

with tf.name_scope('Rnn'):
    outputs,states=build_rnn(X_onehot)
    outputs=tf.transpose(outputs,perm=[1,0,2])
    W=tf.Variable(tf.random_normal((n_neurons,unique_chars),stddev=0.001))
    B=tf.Variable(tf.zeros((1,unique_chars)))
with tf.name_scope('cross_entropy'):
    Ys=tf.matmul(outputs[0],W)+B
    prediction = tf.nn.softmax(Ys)
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_onehot,logits=Ys))
with tf.name_scope('adam_optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    minimize = optimizer.minimize(cross_entropy)
    
mistakes = tf.not_equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
error = tf.reduce_sum(tf.cast(mistakes, tf.float32))

In [26]:
init=tf.global_variables_initializer()

In [27]:
batch_size=100
no_of_batches=int(total_chars//batch_size)
epoch=5000
ptr=0
with tf.Session() as sess:
    sess.run(init)
    for i in range(epoch):
        print("epoch ",i)
        if ptr+batch_size+1 >= len(data): 
            ptr=0
        for j in range(no_of_batches):
            inputs,targets=generate_next_batch(batch_size,ptr)
            ptr+=batch_size
            if(j%100==0):
                loss=sess.run(cross_entropy,feed_dict={X:inputs, Y:targets}) 
                sample_ix=generator(inputs[0],200)
                txt = ''.join(id_char[ix] for ix in sample_ix)
                print('iteration %i epoch %i loss %f'%(j,i,loss))
                print ('----\n %s \n----' % (txt, ))
            sess.run(minimize,feed_dict={X:inputs,Y:targets})

epoch  0
iteration 0 epoch 0 loss 4.143154
----
 pjewyy'Vwl,LzGii;MIA:XApcxZbwY:qmj-v;sBIDnYa?P;b.iLHjdw!oZYTuNzhN.Xvm3HZCklEHmhDVFTm;TErtfDl&QHnbO,s,r;baWR&JmAq!vFVvidMvdTsJOkVCbOGDKjTblhL-WnANY:n'NaWyU3Wg&JEpFFMozTiYwg,bJrqBujXKD!t?Pk.MWoddFKKEdtx 
----
iteration 100 epoch 0 loss 2.800530
----
  out Yow,ARCIUSh t t gonThand the y howsaten,e. ath thy alloway t t IUSete wereeth o m wir'or obey oobrerarnk nomat foucule? tcr ine,As, thon s frENI wery'eYear k Fenatios,Whont, horld thieean thathel 
----
iteration 200 epoch 0 loss 2.481176
----
 or thanorspathe ifreles m IPrsiewomemeroman keaghy n dof, momlllan tha,Sharam arwr w tout In s heott. ik st ag Ravimugas haee uimt LL.Have othur hilllle be o, wlfs, b Ie. l wedsthu awil.ooooo sca wis, 
----
iteration 300 epoch 0 loss 2.419646
----
 unsoonyoneme med,Weruche u t,Wetu ongWhe,TNIDIUS: me thou g pal;Wiu?Ifoton ap atr sed wie; cgWo hequ mismandsere chattoranoreey sisere thawhentuca be ot hore d:Whiemerthapes tougreren st c.LARo med te 
--

iteration 3300 epoch 0 loss 2.468539
----
  r thyo andrimsef beD ngeper e s MOF ront ban dertol.JOF hy wh tel tiscthoy BROHAUNT:CHNRY ile t ue.T:Theearedoulke onchese rofot fithy amyepo thaloa he.Jont LINGAUN acanst s he asbl heteetcthe n ans  
----
iteration 3400 epoch 0 loss 2.070296
----
 osin t c?JO,D nout e bady yswitondeingsdeWistin woutld,Thy gotes his a o pthyoriecindechon te ndl,Thys' Gouke Irsicout I:'s rmireris sthy n t lo tsit, n me blear aghor.KIZoof ceve,AUpethins,ERDitneang 
----
iteration 3500 epoch 0 loss 2.583770
----
 my en n,Thixps thofr tell manoring th ngrnghane,Thard folitrd grow' sin wo s wnene ce, wisot spee'st,THUKI,TE lans t or ty th oundingh Eve:Wesefaleprrest ive her. grmanWh thes avinquegharcf n foe sede 
----
iteration 3600 epoch 0 loss 2.293785
----
  s oveng this s beatacoutazean, m u rd cedon metievend bome GRO, my n har'ethy w my inend Hos sain gin astrnd,E h mao facllesAROKENRabrioureand n f eanone?BatWer ban nepares oong ofya l other'd hout G 
----


iteration 6600 epoch 0 loss 2.736053
----
 , be d tst yot, hel fald indous t, afratiss ll y.KINGUClyomy plllas:S:WAG l be t m inde hino CED:MONGUEd wnde h y bourd ghe Fo hitif theen:Armig f ingend me,-jou RDWhy V:Bum f taress:Thirs y, ik,Turm, 
----
iteration 6700 epoch 0 loss 2.504439
----
 agu Whethtshmy foderdy gel kicoth IV: thfld iknee? weind,Tends e w. iny WAn,An:Fr, orarenrthind d's igiosthe.KIV:Mo fan cholldruroth h Wheenge, ig bes becknck, my My indly, brengagis; h, s, inde cenot 
----
iteration 6800 epoch 0 loss 2.752000
----
 yon myowinrdis VINCo har I madooy rt cle f be: w l ime an y, G l thit s,Thar thit thepland me; othy, ERI.Ducak.Groucue.CEns;ESatoubut ect st;AR:'s! HNCENCllfrs m mak o to t t a ber!CENoulorenigldlardw 
----
iteration 6900 epoch 0 loss 2.513494
----
 vothesuncor, s yongohiangiromu o LOMavelon;Tho c so ashasS:IARNeg til V: thonMES: gout ms vend llotoue:e hamehay theeld s, s s,We hasars vekshoucasbld-ealagourungamegat biret s h, ogican, ooo spe a pe 
----


iteration 9900 epoch 0 loss 2.384072
----
 as burg, foathente th an wifarenrano pova et my,Yonio he, arediale new:Gon;ANTust gin allds, oorind isooonnet nowhoghive herm autentheenve so ave hy cfa by!He f y dsorinebe t t, ond th d k;SI o de'he; 
----
iteration 10000 epoch 0 loss 2.572662
----
 wint. aslf myift arifunt ho'sorowior y:'ser;GRIO:Bie I aist c eld il blintl s the y m s, frer w.I ayr'shester at g br miead wiemil manomise tint waindomy qu wicld thind tas, hol, st da your,A w ht mad 
----
iteration 10100 epoch 0 loss 2.516103
----
 ANIA:Anersiound kntaut t IONIt have,A:ENAnd wiriknd thllot fte asowinchar-enift,I wisten tre anor tolfr, l y thiuth, lsagn aucor wile the, ad.Fove stbunigdl thiancay aw, sckn bethar?TRThard, mit, scal 
----
iteration 10200 epoch 0 loss 2.774546
----
 r.BIO:EO:ORUCHIO:HIO: bed foo, p t MI souchenksindeare ws he s.Geshanonous t bifksththIO:Wesowld me IORUCHIORUCHarr itan.PE tsadd y uinesO, wot, t mat.Tin thessso t be hily, arer een s th thee gou fat 
--

iteration 2400 epoch 1 loss 2.160288
----
 n n bewe g mas sse thicee pay INome therdonead isthitores s n,Encis!Thaghaverd ealouruthre us t useane nto ing'ligserathore,AMatinmpel ond;AMed n d ty:Toor t men rd t cre, thaithe ep okneedother, n d  
----
iteration 2500 epoch 1 loss 2.135020
----
 thales oure, wonk'd titofst m, o lat Yon watoueent nst slldido Burd no,ASBe, me oncerean t memyoomofuily, he lla nsein ldeppe f war be gar ll, vel anof phe gle whathis we lalo be ce lonougoullobyofo-p 
----
iteration 2600 epoch 1 loss 2.074221
----
 HAthttino n; me, my ind, mooditant psbelyovor'denon uryferI ERAmeero thy to w w our, winecuthr,ANG D sARKIII t tce k oreeipprouis t, mowle tisoris moo, fsiknd yaroubyetind?RM:M:STHous mer, t mer.KI yo 
----
iteration 2700 epoch 1 loss 2.357299
----
 e, hest d y beawraithen mats-wes EN f hancllel'dlalonorou nr SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS 
----


...

iteration 9000 epoch 1 loss 2.635278
----
 ofe; r ad Ma ileMar; hueand; ou an e l il's. l rr.ELENANGorot?Fand adivyosir thasorde h, n werursst VI y pspheaned:Itint, hicathofrtally, Manondema u r g yore imus tharseceprrou. dinghe.Frow pou inker 
----
iteration 9100 epoch 1 loss 2.871982
----
 A:VITinou ad uted yotou man'and lprABoutheroman ver thith t checauk s youtot, tey s th sost, t t ffad yTher vorilss,Fguerr musengustestheiste d s.DUSA:Fiff tThes s tithindournouke'By m, oudace faven n 
----
iteration 9200 epoch 1 loss 2.510596
----
 ioous;IO:ABreds pr, mavey que I me ped IO: sotheengoton ts ise, tllO:LLUntlldesdn t th.DUKE seme maberger hitthoursedSCaloneaionddist atieme ad,And:LUCourdO:hin thit! yomovinevifishe, sartiveW: avis o 
----
iteration 9300 epoch 1 loss 2.651538
----
 tcar ostaberacr bu rored onowofo,Sarited oote heverisevour beevinove so orerd d,Ane, harrave gioulisTIO:TIO:Halarung cthimat t:HIO:Sak n hemabavisaknoruper aldissudHe t CE ANCENThue pand irpooulde.My  
----


iteration 1500 epoch 2 loss 2.666659
----
 Istthit mod' bis.Te h mmes d.Mate athes hcof wha ghards ou arnd, w wiutanen thare'ta san mour w beaconmaurs meanf PUSh, orstheTibu. ivandeWingh frorakim at sBear ParTwar t me ad antoy:He ftiseaseriego 
----
iteration 1600 epoch 2 loss 2.485343
----
 ely spinscherodBER:Lownd be,Why, te hathithor heldim th h, sebuthelf hthithr'de fucrsthenc.BENES: he.R:DY flashrss we!ENER:S: mathioowit un s, at rthild, houndeay,ASTENose?Whe ancculathithear eTINUCEv 
----
iteration 1700 epoch 2 loss 2.491141
----
 ace weng t the ch my, t thed CUChed prkirTRovis. es, lt kngequlllcomathimyTht me ses?Gal thalle:He;Ish mWevo'sthatrs ay h d VEFrad fomyIDZAnsenetod mit hat t heveyouskes:MGZArchot fanen t hy.Bildy mel 
----
iteration 1800 epoch 2 loss 2.486683
----
  st, h bly mme in hea thergen meve lou bly amewag the counteno-key tispid t pes ayTE: t'te, hourd tshathe ferif; fathe ysa th trou hese inshe l s thefou tho fulsth avy AR:Ime, f nst haty byon totrenth 
----


iteration 4800 epoch 2 loss 2.501791
----
  ale ller, gofadesINuldesotr f wine;-he firocist saisse m, le lerfow s IOMERTINu, nke, pent tat ay, alfr, qunt.CUTE:CUThest cr we? s g, our!OMENo dess ur miththey toy BEO:I ms t: nch, os m lem, Geirof 
----
iteration 4900 epoch 2 loss 3.017091
----
 t end ommoth.RCuso ca bawh vewirk igd hanap-serel m llyoneeste, whe houlithethid yovesthan s suncas g t ave l h RCUTI cy? atheanoud asesAMEROuplde th' the be CERWe be t t Wht:O:Ge wo mat; rerano thit  
----
iteration 5000 epoch 2 loss 2.457729
----
 ovishal s:It orit, bour fa ayo h;Pl tono?JUThin fld?'d!Whofh!JULI a amsasll g aughes sheTuroue!Juserind the be fide my ay winkousth l!Jund t?Isuculacedo wiminice mb-hishearn m, n, pe t.'dshtind hee wa 
----
iteration 5100 epoch 2 loss 2.553625
----
 aworil e ay, ROLI me fe ave:O,'sane: witert's the,--May misha herare sn I OLar Inaoforsthadllstwinowow haveders t bam, mit g herT:Toind oy soreve e:' momatunongrmudofiser passtellit AUTyoomyomoingisd  
----


iteration 8100 epoch 2 loss 2.460010
----
 youllersau EONe ltacowy w gengh mealil tl CAUS:Cld I I amyor t amy w: watof us asef I tang oman ashangokicondnofamererono oout YCougred t waten?S: le, t lesswocard PUTEON tasongoullothat ffef wide, Re 
----
iteration 8200 epoch 2 loss 2.506921
----
 haiceGesat u camud pldTenthor prenino t cemar illyo inTofrTE fat butley yordopst l ncor qucrive t ust, anoun ned tous athiciosimo wan oilivelle llyofovemue ingler,CLAfre oowonedle altomfe dsore t hang 
----
iteration 8300 epoch 2 loss 2.749051
----
 icuns byommuptioniorronof t ds me ofuls t a, pore I indoredod, omaves hile neseveprorn ant horeARELerdh'IVEDUCERviven n gro ko temee bin s hindilist s y itirdus se pend the,Ms chantu.An tacocIOLean.I  
----
iteration 8400 epoch 2 loss 2.435238
----
 tlline noveyobl Son; nou alin sis llayd ithuchand.IO ilit:OPEE:I hu? ou is thind s:WouTRI br co an overot, beree ton, as hu sir, he vifabut, inom tt OPOMik athe onde u, aver wI tht:Fopuee, tha abu d.P 
----


KeyboardInterrupt: 